In [1]:
from exponential import *
from scipy.optimize import minimize

In [2]:
params=np.load('params.npy',allow_pickle=True)[:,:1]

In [3]:
def f(x,oldN_eigenvalues,oldN_eigenvectors):
    if x[0]<=0 or x[1]<=0 or x[2]<=0:
        return 10000

    newparams=np.append(params,np.expand_dims(np.array(x),1)*mp.mpf(1),axis=1)
    subspace=Subspace(newparams.shape[1])

    subspace.set_N_func(N_func)
    subspace.set_H_func(H_func)

    subspace.set_params(newparams)

    subspace.make_N_mat()

    subspace.make_H_mat()
    subspace.find_N_eigens()
    subspace.make_Y_mat()
    subspace.make_invs_sqrt_beta_mats()
    subspace.make_P_mats()
    subspace.find_P_eigens()
    subspace.find_energy_eigenstates()

    novel_contribution=np.sum(
        subspace.energy_eigenstates[0]**2
        *(
            np.array(subspace.N_mat.tolist())[:,-1]
            -np.sum(
                np.array((oldN_eigenvectors.T*subspace.N_mat[:subspace.dim-1,:subspace.dim]).tolist()).transpose()*(
                np.array((oldN_eigenvectors.T*subspace.N_mat[:subspace.dim-1,subspace.dim-1]).tolist()).flatten()/
                np.array(oldN_eigenvalues.tolist()).flatten())
            ,axis=1)
        )**2
    )
    return -novel_contribution

In [ ]:

for i in range(200):
    print()
    print(params.shape[1],"+1")
    x0=np.abs(np.array([400,1,0.02])*np.exp(np.random.normal(0,3,3)))
    print(x0)

    #first calculate N matrix without the new basis to get an orthogonal basis
    subspace=Subspace(params.shape[1])
    subspace.set_N_func(N_func)
    subspace.set_H_func(H_func)
    subspace.set_params(params)
    subspace.make_N_mat()
    subspace.make_H_mat()
    subspace.find_N_eigens()
    subspace.make_Y_mat()
    subspace.make_invs_sqrt_beta_mats()
    subspace.make_P_mats()
    subspace.find_P_eigens()
    subspace.find_energy_eigenstates()
    subspace.find_energy_levels()
    print("Ground State energy level:",subspace.energy_levels[0])
    #calculate the exectation of the delta 
    expdelta=delta(subspace.energy_eigenstates[0],params)
    print("the expectation of the delta:",expdelta)#
    print()

    res=minimize(f,x0=x0,method='Nelder-Mead',args=(subspace.N_eigenvalues,subspace.N_eigenvectors))
    params=np.append(params,np.expand_dims(np.array(res.x),1)*mp.mpf(1),axis=1)
    print(res)


1 +1
[3.15547552e+02 2.52861884e-02 8.61515788e-04]
Ground State energy level: -402.6369442882511064163101587400076640534815731956119257569504104006216731723916477375418818758261605
the expectation of the delta: 0.3300445271137533573509160543244392115899875427132809097288628991985224010445607263434505686871413419

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: -0.00014334871792306638
             x: [ 4.068e+02  3.572e-01  4.888e-03]
           nit: 128
          nfev: 230
 final_simplex: (array([[ 4.068e+02,  3.572e-01,  4.888e-03],
                       [ 4.068e+02,  3.572e-01,  4.888e-03],
                       [ 4.068e+02,  3.572e-01,  4.888e-03],
                       [ 4.068e+02,  3.572e-01,  4.888e-03]]), array([-1.433e-04, -1.433e-04, -1.433e-04, -1.433e-04]))

2 +1
[1.14131456e+03 1.36862432e+01 2.38623489e-02]
Ground State energy level: -402.6370776739728094648365249342860541681789800831793554843598143570204283

In [ ]:
subspace=Subspace(params.shape[1])

subspace.set_N_func(N_func)
subspace.set_H_func(H_func)

subspace.set_params(params)

subspace.make_N_mat()
subspace.make_H_mat()
subspace.find_N_eigens()
subspace.make_Y_mat()
subspace.make_invs_sqrt_beta_mats()
subspace.make_P_mats()
subspace.find_P_eigens()
subspace.find_energy_eigenstates()
subspace.find_energy_levels()

In [ ]:
print(delta(subspace.energy_eigenstates[0],params))

0.3154757921291970117706134350684211675888993361577230827529620185235552540813737217875634944850308944


In [ ]:
np.save("altparams",params,allow_pickle=True)